In [13]:
import pandas as pd
import numpy as np
import random
import os
import gc
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [14]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [15]:
# 데이터 불러오기
train = pd.read_csv('cal.train.csv')
test = pd.read_csv('cal.test.csv')
sample_submission = pd.read_csv('cal.sample_submission.csv', index_col = 0)

In [16]:
ordinal_features = ['Weight_Status', 'Gender']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(train[feature])
    train[feature] = le.transform(train[feature])

    for label in np.unique(test[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test[feature] = le.transform(test[feature])

In [17]:
train_x = train.drop(['ID', 'Calories_Burned','Weight_Status','Height(Remainder_Inches)','Height(Feet)'], axis = 1)
train_y = train['Calories_Burned']
test_x = test.drop(['ID','Weight_Status','Height(Remainder_Inches)','Height(Feet)'], axis = 1)

In [18]:
def custom_scoring_function(estimator, X, y):
    y_pred = estimator.predict(X)
    return -mean_squared_error(y, np.rint(y_pred))

In [19]:
pipeline = Pipeline([('poly', PolynomialFeatures(degree=3)),
                     ('reduce_dim', SelectKBest(f_regression,k=40)),
                     ('regressor', LinearRegression())])

In [20]:
# 교차 검증 수행
scores = np.sqrt(-cross_val_score(pipeline, train_x,train_y, cv=5, scoring=custom_scoring_function))
scores
# 교차 검증 평균 점수 출력
print('Cross-validation RMSE:', scores.mean()) 

Cross-validation RMSE: 0.13479018967602313


In [21]:
pipeline.fit(train_x, train_y)
test_y_pred = pipeline.predict(test_x)
test_y_pred =np.rint(test_y_pred )
test_y_pred 

array([173., 189.,  53., ..., 130.,  32., 191.])

In [22]:
submission = pd.read_csv('submission_2nd.csv',index_col = 0)
submission['Calories_Burned'] = test_y_pred
submission

,Calories_Burned
ID,
TEST_0000,173.0
TEST_0001,189.0
TEST_0002,53.0
TEST_0003,161.0
TEST_0004,226.0
...,...
TEST_7495,197.0
TEST_7496,10.0
TEST_7497,130.0
